In [56]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date

import pickle

from imblearn.over_sampling import SMOTE

import numpy as np

import torch
from torch import nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import scipy
from torch import optim
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
torch.__version__



'1.2.0'

In [57]:
# first we're going to read in the dataframe. 
# Going to start from scratch and experiment with different transformations here

jeremy_processed_fp = '/home/jeremy/Desktop/projects/data/Annotation/processed_aita_2019_08.pickle'
aita = pd.read_pickle(jeremy_processed_fp)
aita = aita.sample(frac=1).reset_index(drop=True) #forgot what I have this step here for I think it's the numbered index reset

# reduce to just YTA & NTA
aita = aita[aita['link_flair_text'].isin(['Not the A-hole', 'Asshole'])]
aita.groupby('link_flair_text').size().sort_values(ascending = False)


# need to remove reddit post common words.. these are not part of the story but a function of posting on reddit
# also this keeps complaining I think it's depreciated but I'll take a look later
aita['proc_text'] = [x.replace('x200b', '') for x in aita['proc_text']]
aita['proc_text'] = [x.replace('edit', '') for x in aita['proc_text']]
aita['proc_text'] = [x.replace('removed', '') for x in aita['proc_text']]
aita['proc_text'] = [x.replace('deleted', '') for x in aita['proc_text']]
#aita['proc_title'] = [x.replace('x200b', '') for x in aita['proc_title']]

In [58]:
label_id_df = aita[['link_flair_text', 'label_id']].drop_duplicates()

# Dictionaries for future use
label_to_id = dict(label_id_df.values)
id_to_label = dict(label_id_df[['label_id', 'link_flair_text']].values)

print(label_to_id)

{'Asshole': 2, 'Not the A-hole': 0}


In [62]:


x_train, x_test, y_train, y_test = train_test_split(
    aita['proc_text'], aita['label_id'], test_size=.2, stratify=aita['link_flair_text'], random_state=42)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)



(12493,) (3124,) (12493,) (3124,)


def tfidf_transform(train, test):
    kwargs = {
            'ngram_range': (1,1),  # Use 1-grams + 2-grams.
            'analyzer': 'word',  # Split text into word tokens.
            'min_df': 1,
            'stop_words': "english",
    }
    vectorizer = TfidfVectorizer(**kwargs)
    # Learn vocabulary from training texts and vectorize training texts.
    train_transformed = vectorizer.fit_transform(train)
    # Vectorize validation texts.
    test_transformed = vectorizer.transform(test)
    return train_transformed, test_transformed

x_train, x_test = tfidf_transform(x_train, x_test)

In [63]:


vectorizer = TfidfVectorizer()

# Learn vocabulary from training texts and vectorize training texts.
x_train = vectorizer.fit_transform(x_train)

# Vectorize test texts.
x_test = vectorizer.transform(x_test)



In [64]:
x_train = torch.tensor(scipy.sparse.csr_matrix.todense(x_train)).float()
x_test = torch.tensor(scipy.sparse.csr_matrix.todense(x_test)).float()
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

In [65]:
model = nn.Sequential(nn.Linear(x_train.shape[1], 64),
                      nn.ReLU(),
                      nn.Dropout(0.1),
                      nn.Linear(64, aita['link_flair_text'].nunique()),
                      nn.LogSoftmax(dim=1))

# Define the loss
criterion = nn.NLLLoss()

# Forward pass, get our logits
logps = model(x_train)
# Calculate the loss with the logits and the labels
loss = criterion(logps, y_train)

loss.backward()

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.Adam(model.parameters(), lr=0.002)

RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /opt/conda/conda-bld/pytorch_1565272271120/work/aten/src/THNN/generic/ClassNLLCriterion.c:94